<a href="https://colab.research.google.com/github/lchudal89/geomap/blob/main/anpa_members_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing libraries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install geopy 
!pip install Nominatim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for Nominatim: filename=nominatim-0.1-py3-none-any.whl size=2363 sha256=576ee659fb8e4834fa65ed1f21db3caf44c63846976e01a4ed3bfafc71cc1bea
  Stored in directory: /root/.cache/pip/wheels/61/a8/8d/06a142598c8ff2d5ac76ee5c2935f22dcf67d05e208722c4c0
Successfully built Nominatim


In [ ]:
# Nominatim generates longitude, latitude of most of the city,state around the world
from  geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=None)

/usr/local/lib/python3.8/dist-packages/geopy/geocoders/osm.py:132: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  warnings.warn(


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Data science/ANPA-data.csv')
data.head()

,City,Country,State,Current Affiliation,Field of Study/Interest,Level
0,Worcester,US,MA,Worcester State,Physics,Lifetime Membership
1,Pittsburgh,US,PA,NETL,Applied Physics,Lifetime Membership
2,NaN,US,Illinois,Argonne National Laboratory,"Accelerator Physics, Optics, Laser-Plasma, Com...",-
3,Lubbock,US,Texas,"Texas Tech University, CMS experiment at the L...",High Energy Physics,Lifetime Membership
4,Kansas City,US,Missouri,University of Missouri-Kansas City,Solid-state Physics,Lifetime Membership


In [ ]:
data['City']=data['City'].fillna('') # replacing nan with " " was helpful when trying to generate location in next step
data.head()

,City,Country,State,Current Affiliation,Field of Study/Interest,Level
0,Worcester,US,MA,Worcester State,Physics,Lifetime Membership
1,Pittsburgh,US,PA,NETL,Applied Physics,Lifetime Membership
2,,US,Illinois,Argonne National Laboratory,"Accelerator Physics, Optics, Laser-Plasma, Com...",-
3,Lubbock,US,Texas,"Texas Tech University, CMS experiment at the L...",High Energy Physics,Lifetime Membership
4,Kansas City,US,Missouri,University of Missouri-Kansas City,Solid-state Physics,Lifetime Membership


In [ ]:
# combine city, state, and country to create new columns address
data['address']=data['City'].astype(str)+','+data['State'].astype(str)+ ','+data['Country'].astype(str)
data.head()

,City,Country,State,Current Affiliation,Field of Study/Interest,Level,address,Location
0,Worcester,US,MA,Worcester State,Physics,Lifetime Membership,"Worcester,MA,US","(Worcester, Worcester County, Massachusetts, U..."
1,Pittsburgh,US,PA,NETL,Applied Physics,Lifetime Membership,"Pittsburgh,PA,US","(Pittsburgh, Allegheny County, Pennsylvania, U..."
2,,US,Illinois,Argonne National Laboratory,"Accelerator Physics, Optics, Laser-Plasma, Com...",-,",Illinois,US","(Illinois, South Denley Drive, Dallas, Dallas ..."
3,Lubbock,US,Texas,"Texas Tech University, CMS experiment at the L...",High Energy Physics,Lifetime Membership,"Lubbock,Texas,US","(Lubbock, Lubbock County, Texas, United States..."
4,Kansas City,US,Missouri,University of Missouri-Kansas City,Solid-state Physics,Lifetime Membership,"Kansas City,Missouri,US","(Kansas City, Jackson County, Missouri, United..."


In [ ]:
# getting new column with longitude and latitude based off address 
data['Location']=data['address'].apply(lambda x:geolocator.geocode(x))
data.head()

,City,Country,State,Current Affiliation,Field of Study/Interest,Level,address,Location
0,Worcester,US,MA,Worcester State,Physics,Lifetime Membership,"Worcester,MA,US","(Worcester, Worcester County, Massachusetts, U..."
1,Pittsburgh,US,PA,NETL,Applied Physics,Lifetime Membership,"Pittsburgh,PA,US","(Pittsburgh, Allegheny County, Pennsylvania, U..."
2,,US,Illinois,Argonne National Laboratory,"Accelerator Physics, Optics, Laser-Plasma, Com...",-,",Illinois,US","(Illinois, South Denley Drive, Dallas, Dallas ..."
3,Lubbock,US,Texas,"Texas Tech University, CMS experiment at the L...",High Energy Physics,Lifetime Membership,"Lubbock,Texas,US","(Lubbock, Lubbock County, Texas, United States..."
4,Kansas City,US,Missouri,University of Missouri-Kansas City,Solid-state Physics,Lifetime Membership,"Kansas City,Missouri,US","(Kansas City, Jackson County, Missouri, United..."


In [ ]:
# It seems the location columns has returned null for 10 entries.
data.isnull().sum()

City                        0
Country                     0
State                      39
Current Affiliation        34
Field of Study/Interest    34
Level                       0
address                     0
Location                   10
dtype: int64

In [ ]:
data[data['Location'].isnull()]
# There are mismatch in the city, country, and state. For example,  the code didn't find the valid address for "Bedford, Texas,NP". 
# We have 10 such entries where location is returned as None. 
#I will discard these 10 entries for now and use only rest of the valid entries for creating map.

,City,Country,State,Current Affiliation,Field of Study/Interest,Level,address,Location
59,New York NY USA,US,United States,Brooklyn College and The Graduate Center of CUNY,Semiconductor Physics,Student Membership,"New York NY USA,United States,US",None
82,Bedford,NP,Texas,Ericsson,Research Scientist/Artificial Inteligence/ Mac...,Regular Membership,"Bedford,Texas,NP",None
139,Baglung,NP,Gandaki State,Tribhuvan University,"Data Science, Solid State Physics, Astrophysics",-,"Baglung,Gandaki State,NP",None
244,Baltimore,NP,MARYLAND,UMBC,Atmospheric Physics,Student Membership,"Baltimore,MARYLAND,NP",None
245,Baltimore,NP,MARYLAND,UMBC,Atmospheric Physics,First Time Student,"Baltimore,MARYLAND,NP",None
250,Littlerock,US,Arkansas,UALR,Condensed matter physics/ nanotechnology,Student Membership,"Littlerock,Arkansas,US",None
251,Littlerock,US,Arkansas,UALR,Condensed matter physics/ nanotechnology,First Time Student,"Littlerock,Arkansas,US",None
266,Arkansas,US,FAYETEVILLE,University of Arkansas,Physics,First Time Student,"Arkansas,FAYETEVILLE,US",None
329,Omaha,NP,NE,Creighton University,Medical Physics,First Time Student,"Omaha,NE,NP",None
379,Elhhurst,US,New York,Physics Department,Photonics,First Time Student,"Elhhurst,New York,US",None


In [ ]:
# selecting only valid entries
data1=data[data['Location'].notnull()]
data1.head()

,City,Country,State,Current Affiliation,Field of Study/Interest,Level,address,Location
0,Worcester,US,MA,Worcester State,Physics,Lifetime Membership,"Worcester,MA,US","(Worcester, Worcester County, Massachusetts, U..."
1,Pittsburgh,US,PA,NETL,Applied Physics,Lifetime Membership,"Pittsburgh,PA,US","(Pittsburgh, Allegheny County, Pennsylvania, U..."
2,,US,Illinois,Argonne National Laboratory,"Accelerator Physics, Optics, Laser-Plasma, Com...",-,",Illinois,US","(Illinois, South Denley Drive, Dallas, Dallas ..."
3,Lubbock,US,Texas,"Texas Tech University, CMS experiment at the L...",High Energy Physics,Lifetime Membership,"Lubbock,Texas,US","(Lubbock, Lubbock County, Texas, United States..."
4,Kansas City,US,Missouri,University of Missouri-Kansas City,Solid-state Physics,Lifetime Membership,"Kansas City,Missouri,US","(Kansas City, Jackson County, Missouri, United..."


In [ ]:
# creating two new columns for latitude and longitude from the location columns
data1['Latitude']=data1['Location'].apply(lambda x:x.latitude)
data1['Longitude']=data1['Location'].apply(lambda x:x.longitude)
data1.head()

<ipython-input-11-13ea5cb19e71>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Latitude']=data1['Location'].apply(lambda x:x.latitude)
<ipython-input-11-13ea5cb19e71>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Longitude']=data1['Location'].apply(lambda x:x.longitude)


,City,Country,State,Current Affiliation,Field of Study/Interest,Level,address,Location,Latitude,Longitude
0,Worcester,US,MA,Worcester State,Physics,Lifetime Membership,"Worcester,MA,US","(Worcester, Worcester County, Massachusetts, U...",42.262562,-71.801888
1,Pittsburgh,US,PA,NETL,Applied Physics,Lifetime Membership,"Pittsburgh,PA,US","(Pittsburgh, Allegheny County, Pennsylvania, U...",40.441694,-79.990086
2,,US,Illinois,Argonne National Laboratory,"Accelerator Physics, Optics, Laser-Plasma, Com...",-,",Illinois,US","(Illinois, South Denley Drive, Dallas, Dallas ...",32.723180,-96.805191
3,Lubbock,US,Texas,"Texas Tech University, CMS experiment at the L...",High Energy Physics,Lifetime Membership,"Lubbock,Texas,US","(Lubbock, Lubbock County, Texas, United States...",33.563521,-101.879336
4,Kansas City,US,Missouri,University of Missouri-Kansas City,Solid-state Physics,Lifetime Membership,"Kansas City,Missouri,US","(Kansas City, Jackson County, Missouri, United...",39.100105,-94.578142


### Map using Folium

In [ ]:
import folium

In [ ]:
world_map=folium.Map(location=[23.63, -102.35],tiles='Stamen Terrain' ,zoom_start=4)
world_map

In [ ]:
data1.columns

Index(['City', 'Country', 'State', 'Current Affiliation',
       'Field of Study/Interest', 'Level', 'address', 'Location', 'Latitude',
       'Longitude'],
      dtype='object')

In [ ]:
data1.Level.unique() # I want to use as Level feature as color code in the map. Looking at the unique values.

array(['Lifetime Membership', '-', 'Student Membership',
       'Regular Membership', 'First Time Student',
       'General Term Membership'], dtype=object)

In [ ]:
#assigning color to each features 
color_map={'Lifetime Membership':'red','Student Membership':'green',
       'Regular Membership':'blue', 'First Time Student':'orange',
       'General Term Membership':'yellow','-':'gray'}

In [ ]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for index,record in data1.iterrows():
  folium.Marker(location=[record[8],record[9]],popup=record[5],icon=folium.Icon(color=color_map[record[5]],icon='',fill_opacity=0.5)).add_to(incidents)
# add incidents to map
world_map.add_child(incidents)

<ipython-input-83-4c5bfd480dbd>:6: UserWarning: color argument of Icon should be one of: {'gray', 'pink', 'lightgray', 'darkblue', 'cadetblue', 'red', 'darkpurple', 'lightred', 'purple', 'darkred', 'white', 'orange', 'lightgreen', 'beige', 'blue', 'darkgreen', 'lightblue', 'black', 'green'}.
  folium.Marker(location=[record[8],record[9]],popup=record[5],icon=folium.Icon(color=color_map[record[5]],icon='',fill_opacity=0.5)).add_to(incidents)


In [ ]:
# Creating legends
import branca

legend_html = '''
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    bottom: 10px;
    left: 20px;
    width: 250px;
    height: 250px;
    z-index:9999;
    font-size:14px;
    font-weight:bold;
    background-color:white;
    opacity: .85;
    ">
    <p><a style="color:#FF0000 ;font-size:150%;margin-left:20px;">&marker;</a>&emsp;Lifetime Membership</p>
    <p><a style="color:#32CD32;font-size:150%;margin-left:20px;">&marker;</a>&emsp;Student Membership</p>
    <p><a style="color:#00BFFF;font-size:150%;margin-left:20px;">&marker;</a>&emsp;Regular Membership</p>
    <p><a style="color:#FFA500;font-size:150%;margin-left:20px;">&marker;</a>&emsp;First Time Student</p>
    <p><a style="color:#FFFF00;font-size:150%;margin-left:20px;">&marker;</a>&emsp;General Term Membership</p>
    <p><a style="color:#808080;font-size:150%;margin-left:20px;">&marker;</a>&emsp;Unknown</p>
</div>
{% endmacro %}
'''
legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)
folium.LayerControl().add_to(world_map)
world_map.get_root().add_child(legend)
world_map

In [ ]:
world_map.save('Anpa_members.html') # saving html for interactive maps.